# Biosemi Active II recorded file streaming

Resample and Save

In [18]:
import mne
import numpy as np
from mne.channels import make_standard_montage
import matplotlib.pyplot as plt
from mne.datasets import eegbci

path = "C:\\Users\\pipo_\\OneDrive\\Drive_F\\00_Nueron_maze\\00_BCI_ESC\\"
raw = mne.io.read_raw_edf(path+"test_new2.edf", preload=True)

eegbci.standardize(raw)  # set channel names
montage = make_standard_montage("biosemi64")

raw.set_montage(montage, on_missing='ignore')

raw.save(path+"emotiv_pipo.fif", overwrite=True)

Extracting EDF parameters from C:\Users\pipo_\OneDrive\Drive_F\00_Nueron_maze\00_BCI_ESC\test_new2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 11231  =      0.000 ...   350.969 secs...
Overwriting existing file.
Writing C:\Users\pipo_\OneDrive\Drive_F\00_Nueron_maze\00_BCI_ESC\emotiv_pipo.fif
Overwriting existing file.
    Writing channel names to FIF truncated to 15 characters with remapping
Closing C:\Users\pipo_\OneDrive\Drive_F\00_Nueron_maze\00_BCI_ESC\emotiv_pipo.fif
[done]


C:\Users\pipo_\AppData\Local\Temp\ipykernel_17200\3541143535.py:15: RuntimeWarning: This filename (C:\Users\pipo_\OneDrive\Drive_F\00_Nueron_maze\00_BCI_ESC\emotiv_pipo.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(path+"emotiv_pipo.fif", overwrite=True)


[WindowsPath('C:/Users/pipo_/OneDrive/Drive_F/00_Nueron_maze/00_BCI_ESC/emotiv_pipo.fif')]

In [20]:
raw.plot(scalings='auto')

Load data for stream

In [16]:
import mne
import numpy as np
from mne.channels import make_standard_montage
import matplotlib.pyplot as plt
from mne.datasets import eegbci


raw_eeg = mne.io.read_raw_fif(path+"emotiv_pipo.fif")

Opening raw data file C:\Users\pipo_\OneDrive\Drive_F\00_Nueron_maze\00_BCI_ESC\emotiv_pipo.fif...
    Reading extended channel information
    Range : 0 ... 44927 =      0.000 ...   350.992 secs
Ready.


C:\Users\pipo_\AppData\Local\Temp\ipykernel_17200\738832525.py:8: RuntimeWarning: This filename (C:\Users\pipo_\OneDrive\Drive_F\00_Nueron_maze\00_BCI_ESC\emotiv_pipo.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_eeg = mne.io.read_raw_fif(path+"emotiv_pipo.fif")


In [17]:
raw_eeg.plot()

c:\Users\pipo_\miniconda3\envs\MTP_project\Lib\site-packages\mne_qt_browser\_pg_figure.py:4903: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  z = zscore(data, axis=1)


Channels marked as bad:
none
Channels marked as bad:
none


In [ ]:
from pylsl import StreamInfo, StreamOutlet
import time
import mne

events = mne.find_events(raw_eeg, shortest_event = 0, verbose = False) 

eeg_info = StreamInfo(name='Emo_stream', type='EEG', channel_count=raw_eeg.info['nchan'],
                      nominal_srate=raw_eeg.info['sfreq'], channel_format='float32')
eeg_outlet = StreamOutlet(eeg_info)

event_dict =  {'12Hz': 8,
'24Hz': 4,
'6Hz': 10,
'30Hz': 2,
'trial_begin': 1,
'trial_end': 20,
'end_exper': 40
}

id_to_key = {v: k for k, v in event_dict.items()}

def send_eeg_samples(raw_eeg, chunk_size=10):
    """ Send raw EEG data in chunks through LSL. """
    sfreq = raw_eeg.info['sfreq']  # Sampling frequency
    n_samples = len(raw_eeg.times)  # Total number of samples in the raw data
    data, _ = raw_eeg[:, :]  # Get all the EEG data as an array
    index_timestamp = 0


    for idx in range(0, n_samples, chunk_size):
        if idx > events[index_timestamp,0]:
            if  events[index_timestamp,2] in id_to_key:
                print("event", id_to_key[events[index_timestamp,2]])
                index_timestamp += 1 


        # Extract a chunk of `chunk_size` samples
        chunk = data[:, idx:idx + chunk_size]

        # Transpose to match LSL format: list of samples (n_samples x n_channels)
        eeg_outlet.push_chunk(chunk.T.tolist())

        time.sleep(chunk_size / sfreq)  # Sleep to maintain timing


if __name__ == '__main__':
    while True:
        # Send raw EEG data in chunks
        send_eeg_samples(raw_eeg)


event trial_begin
event 24Hz
event trial_end
event trial_begin
event 12Hz
event trial_end
event trial_begin
event 6Hz
event trial_end
event trial_begin
event 24Hz
event trial_end
event trial_begin
event 6Hz
event trial_end
event trial_begin
event 30Hz
event trial_end
event trial_begin
event 6Hz
event trial_end
event trial_begin
event 6Hz
event trial_end
event trial_begin
event 24Hz
event trial_end
event trial_begin
event 6Hz
event trial_end
event trial_begin
event 30Hz
event trial_end
event trial_begin
event 24Hz
event trial_end
event trial_begin
event 30Hz
event trial_end
event trial_begin
event 12Hz
event trial_end
event trial_begin
event 30Hz
event trial_end
event trial_begin
event 6Hz
event trial_end
event trial_begin
event 6Hz


KeyboardInterrupt: 